In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="MonoHime/rubert-base-cased-sentiment-new")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
pipe('вы можете на территорию пронести арбуз')

[{'label': 'NEUTRAL', 'score': 0.8300626277923584}]

In [ ]:
id2label = {0: "NEUTRAL", 1: "POSITIVE", 2: "NEGATIVE"}
label2id = {"NEUTRAL": 0, "POSITIVE": 1, "NEGATIVE": 2}

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("MonoHime/rubert-base-cased-sentiment-new")
model = AutoModelForSequenceClassification.from_pretrained("MonoHime/rubert-base-cased-sentiment-new",num_labels=3,id2label=id2label, label2id=label2id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset

dataset = load_dataset("MonoHime/ru_sentiment_dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/189891 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21098 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd


train_df=dataset['train']
val_df=dataset['validation']

In [ ]:
train_df

Dataset({
    features: ['Unnamed: 0', 'text', 'sentiment'],
    num_rows: 189891
})



Labels meaning
0: NEUTRAL
1: POSITIVE
2: NEGATIVE

In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
!nvidia-smi

Wed May  8 16:18:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
small_train = train_df.shuffle(seed=42).select([i for i in list(range(12000))])
small_val = val_df.shuffle(seed=42).select([i for i in list(range(1000))])
small_train=small_train.rename_column("sentiment", "labels")
small_val=small_val.rename_column("sentiment", "labels")

In [ ]:
small_val

Dataset({
    features: ['Unnamed: 0', 'text', 'labels'],
    num_rows: 1000
})

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True,max_length=512)

tokenized_train = small_train.map(preprocess_function, batched=True)
tokenized_val = small_val.map(preprocess_function, batched=True)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels,average='weighted')["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
!pip install transformers[torch]

  Using cached accelerate-0.30.0-py3-none-any.whl (302 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 k

In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "/drive/MyDrive/"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=1e-5,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=8,
   num_train_epochs=8,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=False,
   logging_steps=50,
   use_cpu=False,
   save_steps=100,
   gradient_accumulation_steps=2
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_val,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)


In [ ]:
!

In [ ]:
import torch

torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss
50,1.121200
100,0.906500
150,0.923500
200,0.944100
250,0.918800
300,0.938800
350,0.844500
400,0.925700
450,0.840000
500,0.803800


Step,Training Loss
50,1.121200
100,0.906500
150,0.923500
200,0.944100
250,0.918800
300,0.938800
350,0.844500
400,0.925700
450,0.840000
500,0.803800


TrainOutput(global_step=12000, training_loss=0.5663842230637868, metrics={'train_runtime': 5873.4193, 'train_samples_per_second': 16.345, 'train_steps_per_second': 2.043, 'total_flos': 1.3916699724721752e+16, 'train_loss': 0.5663842230637868, 'epoch': 8.0})

In [ ]:
model_path = "/content/drive/MyDrive/model/"  # Укажите путь к файлу модели
trainer.save_model(model_path)

In [ ]:
model_path = "/content/drive/MyDrive/model.pth"  # Укажите путь к файлу модели
model.save_pretrained(model_path)

In [ ]:
tokenizer_path = "/content/drive/MyDrive/tokenizer"  # Укажите путь к папке токенизатора
tokenizer.save_pretrained(tokenizer_path)

('/content/drive/MyDrive/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/tokenizer/vocab.txt',
 '/content/drive/MyDrive/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/tokenizer/tokenizer.json')

In [ ]:
trainer.evaluate()


<ipython-input-10-3988a74846a4>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 1.0000849962234497,
 'eval_accuracy': 0.708,
 'eval_f1': 0.7079354501319096,
 'eval_runtime': 28.0551,
 'eval_samples_per_second': 35.644,
 'eval_steps_per_second': 4.456,
 'epoch': 8.0}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sentiment_model = pipeline(model=

In [ ]:
sentence = "хахахах ну ты прям гений.."

# Токенизация предложения
inputs = tokenizer(sentence, return_tensors="pt").to('cuda')

# Получение вывода модели
outputs = model(**inputs)

In [ ]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-3.4442,  0.1157,  2.0436]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
logits = outputs.logits
logits

tensor([[-3.4442,  0.1157,  2.0436]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [ ]:
predicted_class = logits.argmax().item()

In [ ]:
predicted_class

2

In [ ]:
import torch
input_ids = torch.tensor(tokenized_val['input_ids']).unsqueeze(0)  # добавляем размерность пакета
attention_mask = torch.tensor(tokenized_val['attention_mask']).unsqueeze(0)  # добавляем размерность пакета


ValueError: expected sequence of length 88 at dim 1 (got 123)

In [ ]:
predictions = model.predict(tokenized_val)

AttributeError: 'BertForSequenceClassification' object has no attribute 'predict'

In [ ]:
predictions

Инференс

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TextClassificationPipeline
# Путь к папке с вашей моделью
model_path = "/content/drive/MyDrive/model"

# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=3,id2label=id2label, label2id=label2id)
from transformers import pipeline

pipe = pipeline("text-classification", model=model,tokenizer=tokenizer)


In [ ]:
pipe('мда, неприятно')

[{'label': 'NEGATIVE', 'score': 0.574764609336853}]

In [ ]:
import pandas as pd
val_df=pd.DataFrame(tokenized_val)

In [ ]:
predictions = val_df['text'].apply(lambda x: pipe(x,max_length=512))

# Извлекаем только метки из результатов предсказаний
labels = [result['label'] for result in predictions]

# Записываем метки обратно в колонку 'predictions' датафрейма
val_df['predictions'] = labels

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


TypeError: list indices must be integers or slices, not str

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_score)

NameError: name 'y_true' is not defined

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
!pip install google-colab-shell

  Preparing metadata (setup.py) ... done
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4108 sha256=0a5b6e23613705107acc48a780f83b935f757b742a658d44916129991283896e
  Stored in directory: /root/.cache/pip/wheels/f9/36/65/95dd4599be065418a9fe1f482674c8e716ce540f3f484681d2
Successfully built google-colab-shell


In [ ]:
!nvidia-smi

Tue May  7 14:57:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0              27W /  70W |   7265MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--